In [2]:
import pandas as pd

In [3]:
# descobri o mes pelo numero da semana

from datetime import datetime, timedelta
 
def descobre_mes(ano, num_semana):    
    date = str(ano) + '-W' + str(num_semana)
    res = datetime.strptime(date + '-1', "%Y-W%W-%w")
    return res.strftime("%B")

In [4]:
# criação dos charts

import glob

df_charts = pd.DataFrame()

list_years = [2017, 2018, 2019]
for y in list_years:
    year = y
    arquivos = glob.glob('../songs_data/charts/us/' + str(year) +'/*.csv')

    week=0
    for a in arquivos:
        week += 1
        df = pd.read_csv(a, delimiter='\t')
        df['year'] = year
        df['week'] = week
        df_charts = pd.concat([df_charts, df], ignore_index=True)

# df_charts.to_csv('my_data/united_states_charts.csv', index=False)

unindo songs + charts br

In [5]:
import ast

charts_brasil = pd.read_csv('../my_data/charts/brasil_charts.csv')

hits_songs = pd.read_csv('../songs_data/hit_songs/spotify_hits_dataset_complete.csv', delimiter='\t')
# hits_songs['artist_name'] = hits_songs['artist_name'].apply(eval)

hits_songs = hits_songs.explode(['artist_id','artist_name']).reset_index(drop=True)
hits_songs['artist_id'] = hits_songs['artist_id'].apply(ast.literal_eval)
hits_songs['artist_name'] = hits_songs['artist_name'].apply(ast.literal_eval)

print(len(charts_brasil))
print(len(hits_songs))

charts_brasil_songs = pd.merge(charts_brasil, hits_songs, on='song_id')
charts_brasil_songs['month'] = charts_brasil_songs.apply(lambda row: descobre_mes(row['year'], row['week']), axis=1)

print(len(charts_brasil_songs))

# Index(['position', 'song_id', 'song_name_x', 'artist', 'streams',
#        'last_week_position', 'weeks_on_chart', 'peak_position',}
#        'position_status', 'year', 'week', 'song_name_y', 'artist_id',
#        'artist_name', 'popularity', 'explicit', 'song_type', 'track_number',
#        'num_artists', 'num_available_markets', 'release_date', 'duration_ms',
#        'key', 'mode', 'time_signature', 'acousticness', 'danceability',
#        'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness',
#        'valence', 'tempo'],
#       dtype='object')

colunas = ['year', 'week','month', 'song_id', 'song_name_x', 'song_name_y','artist_id', 'artist_name', 'energy', 'danceability']

print(charts_brasil_songs[colunas].head())



31200
13880
31200
   year  week    month                 song_id song_name_x song_name_y   
0  2017     1  January  0EPxmvsG1BY5td4aTOkWBF    Deu Onda    Deu Onda  \
1  2017     2  January  0EPxmvsG1BY5td4aTOkWBF    Deu Onda    Deu Onda   
2  2017     3  January  0EPxmvsG1BY5td4aTOkWBF    Deu Onda    Deu Onda   
3  2017     4  January  0EPxmvsG1BY5td4aTOkWBF    Deu Onda    Deu Onda   
4  2017     5  January  0EPxmvsG1BY5td4aTOkWBF    Deu Onda    Deu Onda   

                  artist_id artist_name  energy  danceability  
0  [3Nipsl6GVwwGyeAk0J29C6]    [MC G15]   0.604         0.931  
1  [3Nipsl6GVwwGyeAk0J29C6]    [MC G15]   0.604         0.931  
2  [3Nipsl6GVwwGyeAk0J29C6]    [MC G15]   0.604         0.931  
3  [3Nipsl6GVwwGyeAk0J29C6]    [MC G15]   0.604         0.931  
4  [3Nipsl6GVwwGyeAk0J29C6]    [MC G15]   0.604         0.931  


In [6]:
artists = pd.read_csv('../songs_data/artists/spotify_artists_info_complete_reduced_genres.csv', delimiter='\t')

charts_song_artist_br = charts_brasil_songs.explode('artist_id').astype({'artist_id': 'str'}).merge(artists, on='artist_id')

charts_song_artist_br.columns

Index(['position', 'song_id', 'song_name_x', 'artist', 'streams',
       'last_week_position', 'weeks_on_chart', 'peak_position',
       'position_status', 'year', 'week', 'song_name_y', 'artist_id',
       'artist_name', 'popularity_x', 'explicit', 'song_type', 'track_number',
       'num_artists', 'num_available_markets', 'release_date', 'duration_ms',
       'key', 'mode', 'time_signature', 'acousticness', 'danceability',
       'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness',
       'valence', 'tempo', 'month', 'name', 'followers', 'popularity_y',
       'genres', 'image_url'],
      dtype='object')

In [13]:
colunas_de_analise = ['song_id', 'song_name_x','genres', 'year', 'month', 'artist', 'energy', 'danceability']
charts_song_artist_br[colunas_de_analise]

,song_id,song_name_x,genres,year,month,artist,energy,danceability
0,0EPxmvsG1BY5td4aTOkWBF,Deu Onda,"['brega funk', 'brazilian funk']",2017,January,MC G15,0.604,0.931
1,0EPxmvsG1BY5td4aTOkWBF,Deu Onda,"['brega funk', 'brazilian funk']",2017,January,MC G15,0.604,0.931
2,0EPxmvsG1BY5td4aTOkWBF,Deu Onda,"['brega funk', 'brazilian funk']",2017,January,MC G15,0.604,0.931
3,0EPxmvsG1BY5td4aTOkWBF,Deu Onda,"['brega funk', 'brazilian funk']",2017,January,MC G15,0.604,0.931
4,0EPxmvsG1BY5td4aTOkWBF,Deu Onda,"['brega funk', 'brazilian funk']",2017,January,MC G15,0.604,0.931
...,...,...,...,...,...,...,...,...
50904,4TnjEaWOeW0eKTKIEvJyCa,Falling,"['r&b', 'rap', 'pop rap']",2019,December,Trevor Daniel,0.430,0.784
50905,4TnjEaWOeW0eKTKIEvJyCa,Falling,"['r&b', 'rap', 'pop rap']",2019,December,Trevor Daniel,0.430,0.784
50906,7wAQTVTggrFQ4bceY6e3GE,Aquecimento do Pedro Sampaio,['brazilian funk'],2019,December,Pedro Sampaio,0.702,0.828
50907,7wAQTVTggrFQ4bceY6e3GE,Aquecimento do Pedro Sampaio,['brazilian funk'],2019,December,Pedro Sampaio,0.702,0.828


In [14]:
import altair as alt

dropdown = alt.binding_select(
    options=[2017, 2018, 2019],
    name='Escolha o ano '
)
xcol_param = alt.param(
    value=2017,
    bind=dropdown
)

alt.Chart(charts_song_artist_br).mark_circle().encode(
    x=alt.X('month:N').title('aaa'),
    y='energy:Q'
    # color='Origin:N'
).transform_calculate(
    x=f'datum[{xcol_param.name}]'
).add_params(
    xcol_param
)

ModuleNotFoundError: No module named 'altair'